Fabrício Carraro - @fabriciocarraro

# RAG - PDF Search in multiple documents using Gemma 2 2B on Colab

This project demonstrates a pipeline for extracting, processing, and querying text data from PDF documents on Google Colab using natural language processing (NLP) techniques and Google's open-source model Gemma 2 2B. The system allows users to input a query, which is then answered based on the content of the PDFs.

<a href="https://colab.research.google.com/github/fabriciocarraro/AI_RAG_PDF-Search-in-multiple-documents_on_Colab/blob/main/AI_RAG_PDF_Search_in_multiple_documents_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
### Select the Colab runtime

To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma 2 setup on Hugging Face

This cookbook uses Gemma 2B instruction tuned model through Hugging Face. So you will need to:

* Get access to Gemma 2 on [huggingface.co](huggingface.co) by accepting the Gemma 2 license on the Hugging Face page of the specific model, i.e., [Gemma 2B IT](https://huggingface.co/google/gemma-2-2b-it).
* Generate a [Hugging Face access token](https://huggingface.co/docs/hub/en/security-tokens) and configure it as a Colab secret 'HF_TOKEN'.

## Retrieval-Augmented Generation (RAG)

Large Language Models (LLMs) can learn new abilities without directly being trained on them. However, LLMs have been known to "hallucinate" when tasked with providing responses for questions they have not been trained on. This is partly because LLMs are unaware of events after training. It is also very difficult to trace the sources from which LLMs draw their responses from. For reliable, scalable applications, it is important that an LLM provides responses that are grounded in facts and is able to cite its information sources.

A common approach used to overcome these constraints is called Retrieval Augmented Generation (RAG), which augments the prompt sent to an LLM with relevant data retrieved from an external knowledge base through an Information Retrieval (IR) mechanism. The knowledge base can be your own corpora of documents, databases, or APIs.

## Installing and importing dependencies

In [ ]:
!pip install -q transformers sentence_transformers faiss-cpu torch PyPDF2 nltk

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
import PyPDF2
import os
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from google.colab import userdata

## Setting up the model and tokenizer

In [ ]:
HUGGING_FACE_ACCESS_TOKEN = userdata.get('HF_TOKEN')

model_name = 'google/gemma-2-2b-it'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    token=HUGGING_FACE_ACCESS_TOKEN
    ).to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_name, token=HUGGING_FACE_ACCESS_TOKEN)

## Extracting and tokenizing info from the PDF files

The `extract_text_from_pdf()` function will look for all PDF files in the `pdf_path` folder.

The `split_text_into_chunks()` function gets the text and breaks it down into smaller chunks.

In [ ]:
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = "".join([page.extract_text() for page in reader.pages])
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

def split_text_into_chunks(text, max_chunk_size=1000):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

## Extracting info from the PDFs

Set the variable `pdf_directory` with the path where your PDF files are. In this case, running on Google Colab, they would be in a folder called `PDFs`, that can be created on the content area on the left side.

A Pandas DataFrame is created containing the path of the corresponding PDF, its chunks and the embedding vector of its chunks.

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Process PDF files
pdf_directory = "/content/PDFs/"
df_documents = pd.DataFrame(columns=['path', 'text_chunks', 'embeddings'])

for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        print(filename)
        pdf_path = os.path.join(pdf_directory, filename)
        text = extract_text_from_pdf(pdf_path)
        chunks = split_text_into_chunks(text)
        document_embeddings = encoder.encode(chunks)
        new_row = pd.DataFrame({'path': [pdf_path], 'text_chunks': [chunks], 'embeddings': [document_embeddings]})
        df_documents = pd.concat([df_documents, new_row], ignore_index=True)

df_documents

## Creating a FAISS index from all document embeddings

Faiss is a library for efficient similarity search and clustering of vectors. The IndexFlatL2 algorithm will be applied to all chunk embedding vectors.

In [ ]:
all_embeddings = np.vstack(df_documents['embeddings'].tolist())
dimension = all_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(all_embeddings)

## Calculating the embedding distance and generating an answer 

The `find_most_similar_chunks()` function will create an embedding vector for your query and compare its similarity to all the chunks it retrieved from the PDF files, returning the most similar one, which will be used as the context for the next function.

The `generate_response()` function will generate an answer using our selected model (Gemma 2 2B) based on the context retrieved from the most similar info chunk.

In [ ]:
def find_most_similar_chunks(query, top_k=3):
    query_embedding = encoder.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = []
    total_chunks = sum(len(chunks) for chunks in df_documents['text_chunks'])
    for i, idx in enumerate(indices[0]):
        if idx < total_chunks:
            doc_idx = 0
            chunk_idx = idx
            while chunk_idx >= len(df_documents['text_chunks'].iloc[doc_idx]):
                chunk_idx -= len(df_documents['text_chunks'].iloc[doc_idx])
                doc_idx += 1
            results.append({
                'document': df_documents['path'].iloc[doc_idx],
                'chunk': df_documents['text_chunks'].iloc[doc_idx][chunk_idx],
                'distance': distances[0][i]
            })
    return results

def generate_response(query, context, max_length=1000):
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')

    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=max_length, num_return_sequences=1)

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extracting the answer part by removing the prompt portion
    answer_start = decoded_output.find("Answer:") + len("Answer:")
    answer = decoded_output[answer_start:].strip()

    return answer

def query_documents(query):
    similar_chunks = find_most_similar_chunks(query)
    context = " ".join([result['chunk'].replace("\n", "") for result in similar_chunks])
    response = generate_response(query, context)
    return response, similar_chunks

## Looking for info in the PDFs

The variable `query` contains the information you want to retrieve from the PDF files.

In [ ]:
query = "How many types of regular Train Car cards are there?"
answer, relevant_chunks = query_documents(query)

print(f"Query: {query}\n\n-----\n")
print(f"Generated answer: {answer}\n\n-----\n")
print("Relevant chunks:")
for chunk in relevant_chunks:
    print(f"Document: {chunk['document']}")
    print(f"Chunk: {chunk['chunk']}".replace("\n", ""))
    print(f"Distance: {chunk['distance']}")
    print()